# Introduction aux réseaux de neurones : Préambule au travail pratique
Matériel de cours rédigé par Pascal Germain, 2019
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Nous vous fournissons un module `office_utils` permettant de charger l'ensemble *Office31* et d'afficher les images qu'il contient.

In [ ]:
from office_utils import charger_office, afficher_grille_office

In [ ]:
charger_office?

Nous vous suggérons de travailler seulement avec les cinq premières classes.

In [ ]:
repertoire_office = '../data/office31/'
classes_office = range(5)

data_x, data_y = charger_office(repertoire_office, classes_office)

Séparons aléatoirement les données en un ensemble d'apprentissage et un ensemble de test de tailles équivalentes (à l'aide des outils de *scikit-learn*).
Nous vous conseillons d'utiliser le même partitionnement des données pour votre projet.

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.25, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

Affichons un échantillon de 24 images sélectionnées aléatoirement dans l'ensemble d'apprentissage. 

In [ ]:
indices_aleatoires = np.random.randint(len(train_y), size=24)
afficher_grille_office(train_x[indices_aleatoires])

## Apprentissage à l'aide d'un réseau de neurones *pleinement connecté*

Comme dans les travaux dirigés, nous utiliserons une classe `ReseauClassifGenerique` pour apprendre notre réseau de neurones. Consultez les commentaires de la classe `ReseauClassifGenerique` pour plus de détails.

In [ ]:
from reseau_classif_generique import ReseauClassifGenerique

In [ ]:
ReseauClassifGenerique?

Un objet `ReseauClassifGenerique` doit être instancié à l'aide d'un `modele` héritant de la classe `torch.nn.Module` (voir le TD de la semaine 4 pour plus de détails). 

Créons une architecture prenant une image en entrée (sous la forme d'un vecteur de $3\times 300 \times 300$ éléments, possédant $5$ sorties (correspondant aux cinq classes d'images) et $20$ neurones sur la couche cachée.

In [ ]:
nb_entrees = 3* 300 * 300
nb_neurones_cachees = 20
nb_sorties = 5

modele_plein = nn.Sequential(
    nn.Linear(nb_entrees, nb_neurones_cachees),
    nn.ReLU(),
    nn.Linear(nb_neurones_cachees, nb_sorties),
    nn.LogSoftmax(dim=1)
)


Exécutons le processus d'apprentissage. Notez que les paramètres de descente en gradient choisis (`eta`, `alpha`, `taille_batch`) ne sont pas nécessairement  optimaux. N'hésitez pas à en suggérer des meilleurs dans votre rapport!

In [ ]:
# Initialisons le réseau de neurones.
reseau_pc = ReseauClassifGenerique(modele_plein, eta=1e-5, alpha=0.01, nb_epoques=500, taille_batch=8, 
                           fraction_validation=.2, patience=20)

# Exécutons l'optimisation
reseau_pc.apprentissage(train_x, train_y)

Vérifions l'acuité du réseau de neurones pleinement connecté sur l'ensemble test. 

In [ ]:
train_pred = reseau_pc.prediction(train_x)
test_pred = reseau_pc.prediction(test_x)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

La précision sur l'ensemble test devrait se situer entre de 60% et 70%, selon les aléas de la descente en gradient stochastique. Vous pouvez répéter l'expérience en exécutant les deux dernières cellules de code.

## Calculer le nombre de paramètres du modèle

Dans l'énoncé du projet, nous vous demandons de tenir compte du nombre de paramètres que votre réseau de neurones doit optimiser. Nous vous fournissons ici une fonction `compter_parametres` qui parcourt les structures de données de pyTorch pour obtenir ce nombre de paramètres, et ainsi valider votre calcul.

In [ ]:
def compter_parametres(modele):
    """Calcule le nombre de paramètres à optimiser dans l'architecture d'un réseau"""
    somme = 0
    for params in modele.parameters():
        nb = 1
        for dimension in params.shape:
            nb *= dimension
        somme += nb
        
    return somme

In [ ]:
compter_parametres(modele_plein)

**Notez bien:** Votre rapport ne doit pas seulement indiquer le total du nombre de paramètres à optimiser, mais détailler la répartition des paramètres pour chaque couche, en tenant compte de l'architecture de votre réseau.

Ainsi, l'architecture pleinement connectée représentée par l'objet `archi_pc` contient $5\,400\,125$ paramètres, ce qui correspond au total des:
* Couche cachée: $[270\,000 \mbox{ entrées}] \times [20 \mbox{ neurones}] + [20 \mbox{ valeurs de biais}] = 5\,400\,020 \mbox{ paramètres}.$
* Couche de sortie: $[20 \mbox{ entrées}] \times [5 \mbox{ neurones}] + [5 \mbox{ valeurs de biais}] = 105 \mbox{ paramètres}.$

In [ ]:
(3*300*300)*20+20 + 20*5+5  